In [ ]:
from google.colab import files

uploaded = files.upload()

Saving adoption_history.csv to adoption_history.csv
Saving pet_adoption_data.csv to pet_adoption_data.csv
Saving synthetic_user_data.csv to synthetic_user_data.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load datasets
pet_df = pd.read_csv('content/pet_adoption_data.csv')
user_df = pd.read_csv('content/synthetic_user_data.csv')
adoption_df = pd.read_csv('content/adoption_history.csv')

def Preprocess_data(pet_df, user_df):
    label_encoders = {}
    categorical_cols = ['PetType', 'Breed', 'Size', 'HealthCondition']

    for col in categorical_cols:
        label_encoders[col] = LabelEncoder()
        pet_df[col] = label_encoders[col].fit_transform(pet_df[col].astype(str))

    user_categorical_cols = ['PreferredPetType', 'LivingSpace', 'Allergies', 'ActivityLevel', 'PastPetExperience']
    for col in user_categorical_cols:
        label_encoders[col] = LabelEncoder()
        user_df[col] = label_encoders[col].fit_transform(user_df[col].astype(str))

    scaler = StandardScaler()
    numeric_cols = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']
    pet_df[numeric_cols] = scaler.fit_transform(pet_df[numeric_cols])

    return pet_df, user_df, label_encoders, scaler


def Cosine_similarity(user_id, pet_df, user_df, adoption_df, label_encoders, scaler):
    if user_id not in user_df['UserID'].values:
        return "User not found"

    user_preferences = user_df[user_df['UserID'] == user_id].iloc[0]
    adopted_pets = adoption_df[adoption_df['UserID'] == user_id]['PetID'].values

    available_pets = pet_df[~pet_df['PetID'].isin(adopted_pets)].copy()

    user_vector = np.array([user_preferences[col] for col in ['PreferredPetType', 'LivingSpace', 'Allergies', 'ActivityLevel', 'PastPetExperience']])
    pet_vectors = available_pets[['PetType', 'Size', 'HealthCondition', 'AgeMonths', 'WeightKg']].values

    user_vector = user_vector.reshape(1, -1)
    similarity_scores = cosine_similarity(user_vector, pet_vectors)[0]

    available_pets.loc[:, 'SimilarityScore'] = similarity_scores
    recommended_pets = available_pets.sort_values(by='SimilarityScore', ascending=False).head(5)

    return recommended_pets



def Rule_Based(user_id, pet_df, user_df, adoption_df, label_encoders, scaler):
    if user_id not in user_df['UserID'].values:
        return "User not found"

    user_preferences = user_df[user_df['UserID'] == user_id].iloc[0]
    adopted_pets = adoption_df[adoption_df['UserID'] == user_id]['PetID'].values

    # Create a user vector
    user_vector = {col: user_preferences[col] for col in ['PreferredPetType', 'LivingSpace', 'Allergies', 'ActivityLevel', 'PastPetExperience']}

    # Filter available pets
    available_pets = pet_df[~pet_df['PetID'].isin(adopted_pets)]

    # Compute similarity based on preferences
    scores = []
    for _, pet in available_pets.iterrows():
        score = 0
        pet_type = label_encoders['PetType'].inverse_transform([int(pet['PetType'])])[0]
        preferred_pet_type = label_encoders['PreferredPetType'].inverse_transform([int(user_vector['PreferredPetType'])])[0]

        if pet_type == preferred_pet_type:
            score += 1  # Match pet type preference

        if pet['Size'] == user_vector['LivingSpace']:
            score += 0.5  # Match living space

        scores.append((pet, score))  # Store pet details instead of just the ID

    # Sort by score and return top recommendations
    scores.sort(key=lambda x: x[1], reverse=True)
    return [pet for pet, _ in scores[:5]]  # Return full pet details



# Preprocess data
pet_df, user_df, label_encoders, scaler = Preprocess_data(pet_df, user_df)


# Example usage
user_id = 1

recommended_pets = Cosine_similarity(user_id, pet_df, user_df, adoption_df, label_encoders, scaler)

# Display recommendations
print(f"\nRecommended pets for User {user_id}:")
if not recommended_pets.empty:
    for _, pet_details in recommended_pets.iterrows():
        print("\n" + "-" * 40)
        print(f"🐾 Pet ID: {pet_details['PetID']}")
        print(f"📌 Type: {label_encoders['PetType'].inverse_transform([int(pet_details['PetType'])])[0]}")
        print(f"🦴 Breed: {label_encoders['Breed'].inverse_transform([int(pet_details['Breed'])])[0]}")
        print(f"📏 Size: {label_encoders['Size'].inverse_transform([int(pet_details['Size'])])[0]}")
        print(f"❤️ Health Condition: {label_encoders['HealthCondition'].inverse_transform([int(pet_details['HealthCondition'])])[0]}")
        print(f"📅 Age (months): {scaler.inverse_transform([[pet_details['AgeMonths'], 0, 0, 0]])[0][0]:.1f}")
        print(f"⚖️ Weight (kg): {scaler.inverse_transform([[0, pet_details['WeightKg'], 0, 0]])[0][1]:.2f}")
        print(f"📊 Similarity Score: {pet_details['SimilarityScore']:.4f}")

            # Boolean values & adoption likelihood
        vaccinated_status = 'Yes' if pet_details['Vaccinated'] else 'No'
       # previous_owner = pet_details['PreviousOwner'] if pet_details['PreviousOwner'] else 'Unknown'
        adoption_likelihood = f"{pet_details['AdoptionLikelihood']}%"

        print(f"💉 Vaccinated: {vaccinated_status}")
      #  print(f"👤 Previous Owner: {previous_owner}")

else:
    print("❌ No valid pet recommendations found.")


Recommended pets for User 1:

----------------------------------------
🐾 Pet ID: 2072
📌 Type: Rabbit
🦴 Breed: Rabbit
📏 Size: Small
❤️ Health Condition: 0
📅 Age (months): 90.0
⚖️ Weight (kg): 14.23
📊 Similarity Score: 0.9910
💉 Vaccinated: No

----------------------------------------
🐾 Pet ID: 644
📌 Type: Rabbit
🦴 Breed: Rabbit
📏 Size: Medium
❤️ Health Condition: 0
📅 Age (months): 96.0
⚖️ Weight (kg): 16.06
📊 Similarity Score: 0.9896
💉 Vaccinated: No

----------------------------------------
🐾 Pet ID: 1433
📌 Type: Rabbit
🦴 Breed: Rabbit
📏 Size: Medium
❤️ Health Condition: 0
📅 Age (months): 93.0
⚖️ Weight (kg): 15.02
📊 Similarity Score: 0.9896
💉 Vaccinated: Yes

----------------------------------------
🐾 Pet ID: 1168
📌 Type: Rabbit
🦴 Breed: Rabbit
📏 Size: Medium
❤️ Health Condition: 0
📅 Age (months): 92.0
⚖️ Weight (kg): 13.71
📊 Similarity Score: 0.9871
💉 Vaccinated: Yes

----------------------------------------
🐾 Pet ID: 2331
📌 Type: Rabbit
🦴 Breed: Rabbit
📏 Size: Small
❤️ Health Condit